<a href="https://colab.research.google.com/github/SHJewell/Lets_Do_Some_Stats/blob/master/chat_gpt_invented_optimizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import math
import random

def ackley(x):
    """
    A common benchmark function:
    f(x) = -20 * exp(-0.2 * sqrt(0.5 * (x1^2 + x2^2)))
           - exp(0.5*(cos(2*pi*x1) + cos(2*pi*x2))) + e + 20
    Generalizable to n dimensions:
    """
    x = np.array(x)
    n = len(x)
    return -20.0 * math.exp(-0.2 * math.sqrt(np.sum(x**2)/n)) \
           - math.exp(np.sum(np.cos(2.0*math.pi*x))/n) + 20 + math.e

def random_point(lower_bounds, upper_bounds, dim):
    """ Generate a random point within the given bounds. """
    return [
        random.uniform(lower_bounds[i], upper_bounds[i])
        for i in range(dim)
    ]

def random_local_search(best_point, lower_bounds, upper_bounds, step_size=0.1):
    """
    Simple local search: pick a random direction near `best_point`.
    """
    dim = len(best_point)
    candidate = []
    for i in range(dim):
        # Propose a new coordinate in a small neighborhood
        perturbation = random.uniform(-step_size, step_size)
        new_coord = best_point[i] + perturbation
        # Clip to boundaries
        new_coord = max(lower_bounds[i], min(upper_bounds[i], new_coord))
        candidate.append(new_coord)
    return candidate

def global_random_search(lower_bounds, upper_bounds, dim):
    """
    Pure global random sample in the domain.
    """
    return random_point(lower_bounds, upper_bounds, dim)

def bandit_optimizer(objective_func,
                     dim=2,
                     lower_bounds=None,
                     upper_bounds=None,
                     n_init=5,      # number of initial random points
                     max_iter=50,   # total number of bandit steps
                     step_size=0.1, # local search step size
                     epsilon=0.1    # epsilon-greedy param
                    ):
    """
    A toy meta-heuristic that adaptively selects among sub-optimizers:
    - random_local_search
    - global_random_search

    For demonstration, we only define two sub-optimizers.
    You could add more, e.g. calling scikit-optimize or nevergrad.
    """
    if lower_bounds is None:
        lower_bounds = [-5] * dim
    if upper_bounds is None:
        upper_bounds = [ 5] * dim

    # List of sub-optimizers (bandit arms)
    sub_optimizers = [random_local_search, global_random_search]
    # We'll track total improvement each sub-optimizer has given us
    improvements = [0.0 for _ in sub_optimizers]
    # We'll track how many times each sub-optimizer was used
    usage_count = [1 for _ in sub_optimizers]  # start from 1 to avoid zero division

    # 1) Initialization
    best_point = None
    best_value = float('inf')
    dim = len(lower_bounds)
    population = [
        random_point(lower_bounds, upper_bounds, dim)
        for _ in range(n_init)
    ]

    for pt in population:
        val = objective_func(pt)
        if val < best_value:
            best_value = val
            best_point = pt

    # 2) Iterations
    for iteration in range(max_iter):
        # --- Bandit selection strategy (epsilon-greedy for simplicity) ---
        if random.random() < epsilon:
            # explore
            chosen_idx = random.randint(0, len(sub_optimizers)-1)
        else:
            # exploit the arm with the highest average improvement
            avg_improvements = [
                improvements[i] / usage_count[i] for i in range(len(sub_optimizers))
            ]
            chosen_idx = int(np.argmax(avg_improvements))

        chosen_sub_optimizer = sub_optimizers[chosen_idx]

        # 3) Propose a new candidate
        if chosen_sub_optimizer == random_local_search:
            candidate = chosen_sub_optimizer(best_point, lower_bounds, upper_bounds, step_size)
        else:
            candidate = chosen_sub_optimizer(lower_bounds, upper_bounds, dim)

        candidate_val = objective_func(candidate)

        # 4) Update best
        old_best = best_value
        if candidate_val < best_value:
            best_value = candidate_val
            best_point = candidate

        improvement = max(0.0, old_best - best_value)
        improvements[chosen_idx] += improvement
        usage_count[chosen_idx] += 1

    return best_point, best_value

def main():
    # Example usage on Ackley function, 2D
    best_x, best_f = bandit_optimizer(ackley, dim=2)
    print("Best solution found:", best_x)
    print("Objective value:", best_f)

if __name__ == "__main__":
    main()


Best solution found: [0.9518506513821077, 0.014417926683716428]
Objective value: 2.5856423309448826
